<a href="https://colab.research.google.com/github/jayU000/python202206/blob/main/pythonbasic12_MachineLearning_titanic3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
# G 드라이브 연결
from google.colab import drive
drive.mount('/gdrive/')

# 모듈 설정
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns


# https://dacon.io/competitions/open/235539/data

train = pd.read_csv('/gdrive/My Drive/Colab Notebooks/titanic/train.csv')
test = pd.read_csv('/gdrive/My Drive/Colab Notebooks/titanic/test.csv')
submission = pd.read_csv('/gdrive/My Drive/Colab Notebooks/titanic/submission.csv')


# 데이터 결합 (위 아래 방향으로 결합)

# 분석을 끝내고 학습을 시킬때 다시 분리 하기 편하도록 split  열을 만들어 각각 이름을 부여한다. 
train['Split'] = 'Train'
test['Split'] = 'Test'

data = pd.concat([train, test], axis=0)

data_num = data.loc[:,['Pclass','Age','SibSp','Parch','Fare','Survived']] 

# age  평균
data_num['Age'] = data_num['Age'].fillna(data_num['Age'].mean())

# fare 최빈
data_num['Fare'] = data_num['Fare'].fillna(data_num['Fare'].mode()[0])



Drive already mounted at /gdrive/; to attempt to forcibly remount, call drive.mount("/gdrive/", force_remount=True).


In [51]:
# 피처 엔지니어링 - 각 피처 열을 모델 학습에 맞게 전처리하고 data['Sex']
print(data['Sex'])

# female 0  male 1
data.loc[data['Sex']=='female','Sex'] = 0
data.loc[data['Sex']=='male','Sex'] = 1

print(data['Sex'])

# 정수형으로 형변환
data['Sex'] = data['Sex'].astype(int)
data['Sex']

0        male
1      female
2      female
3      female
4        male
        ...  
413      male
414    female
415      male
416      male
417      male
Name: Sex, Length: 1309, dtype: object
0      1
1      0
2      0
3      0
4      1
      ..
413    1
414    0
415    1
416    1
417    1
Name: Sex, Length: 1309, dtype: object


0      1
1      0
2      0
3      0
4      1
      ..
413    1
414    0
415    1
416    1
417    1
Name: Sex, Length: 1309, dtype: int64

In [52]:
# Name
print(data['Name'])

# 시리즈 데이터형을 str로 변환하여 자르고 인덱싱
# title = title_name.str.split(",", expand=True)[0]         # 오류 발생 코드
                                                            # 오류 발생 원인 : "," -> 쉼표 뒤에 띄어쓰기가 빠졌음...

title_name = data['Name'].str.split(", ", expand=True)[1]
# 띄어 쓰기 주의... : 쉼표 뒤에 한칸 있음


# 시리즈 데이터형을 str로 변환하여 자르고 인덱싱
title = title_name.str.split(".", expand=True)[0]
print(title.value_counts(dropna=False))


# 비슷한 속성을 가진 타이틀 들을 그룹으로 묶자. 
title = title.replace(['Ms'], 'Miss')
title = title.replace(['Mlle', 'the Countess', 'Lady', 'Don', 'Dona', 'Mme', 'Sir', 'Jonkheer'], 'Noble')
title = title.replace(['Col', 'Major', 'Capt'], 'Officer')
title = title.replace(['Dr', 'Rev'], 'Priest')
data['Title'] = np.array(title)
print(data['Title'].value_counts(dropna=False))


# Title 열을 만들었으므로 필요 없어진 name 열은 삭제한다. 
data= data.drop('Name', axis=1)
data

0                                Braund, Mr. Owen Harris
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                 Heikkinen, Miss. Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                               Allen, Mr. William Henry
                             ...                        
413                                   Spector, Mr. Woolf
414                         Oliva y Ocana, Dona. Fermina
415                         Saether, Mr. Simon Sivertsen
416                                  Ware, Mr. Frederick
417                             Peter, Master. Michael J
Name: Name, Length: 1309, dtype: object
Mr              757
Miss            260
Mrs             197
Master           61
Rev               8
Dr                8
Col               4
Mlle              2
Major             2
Ms                2
Lady              1
Sir               1
Mme               1
Don               1
Capt              1
the Countess      1
Jonkheer     

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Split,Title
0,1,0.0,3,1,22.0,1,0,A/5 21171,7.2500,NaN,S,Train,Mr
1,2,1.0,1,0,38.0,1,0,PC 17599,71.2833,C85,C,Train,Mrs
2,3,1.0,3,0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Train,Miss
3,4,1.0,1,0,35.0,1,0,113803,53.1000,C123,S,Train,Mrs
4,5,0.0,3,1,35.0,0,0,373450,8.0500,NaN,S,Train,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,1,NaN,0,0,A.5. 3236,8.0500,NaN,S,Test,Mr
414,1306,NaN,1,0,39.0,0,0,PC 17758,108.9000,C105,C,Test,Noble
415,1307,NaN,3,1,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Test,Mr
416,1308,NaN,3,1,NaN,0,0,359309,8.0500,NaN,S,Test,Mr


In [53]:
# 나이대
bins = [0, 4, 8, 12, 16, 32, 36, 48, 56, 64, 100]
labels = ['Infant','Child1','Chind2','Youth1','Youth2','Adult1','Adult2','Middle Aged','Senior','Elderly']

data['AgeBin'] = pd.cut(data['Age'], bins=bins, labels=labels)
print(data)

data_ = data.loc[data['Split']=='Train', :]
data_.groupby('AgeBin')['Survived'].mean()

     PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch  \
0              1       0.0       3    1  22.0      1      0   
1              2       1.0       1    0  38.0      1      0   
2              3       1.0       3    0  26.0      0      0   
3              4       1.0       1    0  35.0      1      0   
4              5       0.0       3    1  35.0      0      0   
..           ...       ...     ...  ...   ...    ...    ...   
413         1305       NaN       3    1   NaN      0      0   
414         1306       NaN       1    0  39.0      0      0   
415         1307       NaN       3    1  38.5      0      0   
416         1308       NaN       3    1   NaN      0      0   
417         1309       NaN       3    1   NaN      1      1   

                 Ticket      Fare Cabin Embarked  Split   Title  AgeBin  
0             A/5 21171    7.2500   NaN        S  Train      Mr  Youth2  
1              PC 17599   71.2833   C85        C  Train     Mrs  Adult2  
2      STON/O2. 31012

AgeBin
Infant         0.675000
Child1         0.642857
Chind2         0.266667
Youth1         0.483871
Youth2         0.369942
Adult1         0.479452
Adult2         0.356522
Middle Aged    0.466667
Senior         0.375000
Elderly        0.090909
Name: Survived, dtype: float64

In [54]:
# 요금에 1개의 결측치 처리

data.loc[data['Fare'].isnull(),:]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Split,Title,AgeBin
152,1044,NaN,3,1,60.5,0,0,3701,NaN,NaN,S,Test,Mr,Senior


In [55]:
# 요금과 객실 등급의 상관 관계는 아주 높기 때문에 3등석 객실 승객의 탑승 요금의 평균값을 이용하여 
# 결측치를 채운다. 
# 중요한 점은 전체 요금의 평균값이 아닌 3등급만의 평균을 계산하여 채운다. 

p3_fare_mean = data.loc[data['Pclass']==3,'Fare'].mean()

data['Fare'] = data['Fare'].fillna(p3_fare_mean)

data.loc[data['Fare'].isnull(),:]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Split,Title,AgeBin


In [68]:
data.loc[data['Embarked'].isnull(),:]
data['Fare'] = data['Fare'].fillna('C')
data_.groupby('Embarked')['Survived'].mean()

Embarked
C    0.553571
Q    0.389610
S    0.336957
Name: Survived, dtype: float64

In [69]:
onehot_prefix = []
for col in ['Embarked']:
  data[col] = data[col].astype('category')    # 그냥 문자열 데이터가 아닌 카테고리로 형변환 
  data = pd.get_dummies(data, columns = [col], prefix = col[:3], drop_first= False)
  onehot_prefix.append(col[:3])

data_pre = data.loc[:, [col for col in data.columns if col[:3] in onehot_prefix]].head()
# data= data.drop('Age', axis=1)
# data
data_pre

,Emb_C,Emb_Q,Emb_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1


In [56]:
# 원핫 인코딩 # 
# 문자열 데이터를 숫자형 데이터로 변경하는 또 다른 방법 

onehot_prefix = []
for col in ['Title', 'AgeBin', 'Embarked']:
  data[col] = data[col].astype('category')    # 그냥 문자열 데이터가 아닌 카테고리로 형변환 
  data = pd.get_dummies(data, columns = [col], prefix = col[:3], drop_first= False)
  onehot_prefix.append(col[:3])

data_pre = data.loc[:, [col for col in data.columns if col[:3] in onehot_prefix]].head()
# data= data.drop('Age', axis=1)
# data
data_pre

,Age,Tit_Master,Tit_Miss,Tit_Mr,Tit_Mrs,Tit_Noble,Tit_Officer,Tit_Priest,Age_Infant,Age_Child1,Age_Chind2,Age_Youth1,Age_Youth2,Age_Adult1,Age_Adult2,Age_Middle Aged,Age_Senior,Age_Elderly
0,22.0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,38.0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
2,26.0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,35.0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
4,35.0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [57]:
data_pre= data_pre.drop('Age', axis=1)
print(data_pre)
print(data)


   Tit_Master  Tit_Miss  Tit_Mr  Tit_Mrs  Tit_Noble  Tit_Officer  Tit_Priest  \
0           0         0       1        0          0            0           0   
1           0         0       0        1          0            0           0   
2           0         1       0        0          0            0           0   
3           0         0       0        1          0            0           0   
4           0         0       1        0          0            0           0   

   Age_Infant  Age_Child1  Age_Chind2  Age_Youth1  Age_Youth2  Age_Adult1  \
0           0           0           0           0           1           0   
1           0           0           0           0           0           0   
2           0           0           0           0           1           0   
3           0           0           0           0           0           1   
4           0           0           0           0           0           1   

   Age_Adult2  Age_Middle Aged  Age_Senior  Age_Elderly 

In [70]:
# 피처 스케일링 - 모델 학습에 사용할 피처의 스케일을 0 ~ 1 범위로 정규화 처리한다. 
data_num = data.loc[:,['Emb_C', 'Emb_Q',	'Emb_S', 'Tit_Master', 'Tit_Miss', 'Tit_Mr', 'Tit_Mrs', 'Tit_Noble', 'Tit_Officer', 'Tit_Priest', 'Age_Infant', 'Age_Child1', 'Age_Chind2', 'Age_Youth1', 'Age_Youth2', 'Age_Adult1', 'Age_Adult2', 'Age_Middle Aged', 'Age_Senior', 'Age_Elderly', 'PassengerId','Survived','Pclass','Sex','Fare','Split','SibSp', 'Parch']]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaled_cols = [col for col in data_num.loc[:, 'Pclass':]. columns if col != 'Split']
data_scaled = data_num.loc[:, scaled_cols]
data_scaled = scaler.fit_transform(data_scaled)

data_scaled

array([[1.        , 1.        , 0.01415106, 0.125     , 0.        ],
       [0.        , 0.        , 0.13913574, 0.125     , 0.        ],
       [1.        , 0.        , 0.01546857, 0.        , 0.        ],
       ...,
       [1.        , 1.        , 0.01415106, 0.        , 0.        ],
       [1.        , 1.        , 0.01571255, 0.        , 0.        ],
       [1.        , 1.        , 0.0436405 , 0.125     , 0.11111111]])

In [71]:
data_num.loc[:, scaled_cols] = data_scaled[:,:]
data_num

,Emb_C,Emb_Q,Emb_S,Tit_Master,Tit_Miss,Tit_Mr,Tit_Mrs,Tit_Noble,Tit_Officer,Tit_Priest,...,Age_Senior,Age_Elderly,PassengerId,Survived,Pclass,Sex,Fare,Split,SibSp,Parch
0,0,0,1,0,0,1,0,0,0,0,...,0,0,1,0.0,1.0,1.0,0.014151,Train,0.125,0.000000
1,1,0,0,0,0,0,1,0,0,0,...,0,0,2,1.0,0.0,0.0,0.139136,Train,0.125,0.000000
2,0,0,1,0,1,0,0,0,0,0,...,0,0,3,1.0,1.0,0.0,0.015469,Train,0.000,0.000000
3,0,0,1,0,0,0,1,0,0,0,...,0,0,4,1.0,0.0,0.0,0.103644,Train,0.125,0.000000
4,0,0,1,0,0,1,0,0,0,0,...,0,0,5,0.0,1.0,1.0,0.015713,Train,0.000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0,0,1,0,0,1,0,0,0,0,...,0,0,1305,NaN,1.0,1.0,0.015713,Test,0.000,0.000000
414,1,0,0,0,0,0,0,1,0,0,...,0,0,1306,NaN,0.0,0.0,0.212559,Test,0.000,0.000000
415,0,0,1,0,0,1,0,0,0,0,...,0,0,1307,NaN,1.0,1.0,0.014151,Test,0.000,0.000000
416,0,0,1,0,0,1,0,0,0,0,...,0,0,1308,NaN,1.0,1.0,0.015713,Test,0.000,0.000000


In [72]:
selected_features = ['Pclass','Sex','Fare','Tit_Master', 'Tit_Miss', 'Tit_Mr', 'Tit_Mrs', 'Tit_Noble', 'Tit_Officer', 'Tit_Priest', 'Age_Infant', 'Age_Child1', 'Age_Chind2', 'Age_Youth1', 'Age_Youth2', 'Age_Adult1', 'Age_Adult2', 'Age_Middle Aged', 'Age_Senior', 'Age_Elderly', 'SibSp', 'Parch']

y_train = data_num.loc[data_num['Split']=='Train', 'Survived']
X_train = data_num.loc[data_num['Split']=='Train', selected_features]
X_test = data_num.loc[data_num['Split']=='Test', selected_features]

print('Train 데이터 셋 크기 : ', X_train.shape, y_train.shape)
print('Test 데이터 셋 크기 : ', X_test.shape)

Train 데이터 셋 크기 :  (891, 22) (891,)
Test 데이터 셋 크기 :  (418, 22)


In [124]:
# 훈련용과 검증용 8:2 로 분할

from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.1, shuffle=True, random_state=20)


In [126]:
# voting
from sklearn.ensemble import VotingClassifier as Vot 
from sklearn.tree import DecisionTreeClassifier as DT 
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier as KN   
from sklearn.ensemble import RandomForestClassifier 
from xgboost import XGBClassifier as XGBC 

xgbc = XGBC(n_estimators=100, max_depth=4, random_state=10) 
xgbc.fit(X_tr, y_tr) 

rfc = RandomForestClassifier(n_estimators=100, max_depth=4, random_state=10) 
rfc.fit(X_tr, y_tr)

knn = KN(n_neighbors=4) 
knn.fit(X_tr, y_tr)     

dtc = DT(max_depth = 3, random_state = 20)
dtc.fit(X_tr, y_tr)

evc = Vot(estimators=[('XGBC', xgbc),('RFC', rfc),('KNN', knn),('DT', dtc)],   
          voting='hard')   
evc.fit(X_tr, y_tr) 

# 예측 (시험)
evc_pred = evc.predict(X_val)           
evc_pred 

# 평가
from sklearn.metrics import accuracy_score  
accuracy_score(y_val, evc_pred)

# 0.8770949720670391        강사님 정확도
# 0.88268156424581          5, 2, 3
# 0.8888888888888888        9:1
# 0.9                       xgbc(100,5,10) 추가
# 0.9                       xgbc(100,4,10) 추가

0.9

In [127]:
# 제출용 파일 만들기

y_test_pred = dtc.predict(X_test)
submission['Survived'] = y_test_pred.astype(int)

submission.to_csv('/gdrive/My Drive/Colab Notebooks/titanic/submission_008.csv', index=False)